In [1]:
import pandas as pd
import re 
import numpy
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import * 
from pyspark.sql.functions import length
from pyspark.sql.functions import udf
from pyspark.sql import functions as f
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import col, when
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer


In [2]:
spark=SparkSession.builder.appName('mmarulandc').getOrCreate()
csv = '/FileStore/tables/articles1.csv'
df1 = spark.read.csv(csv).toPandas()
df1.head(2)

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9
0,None,id,title,publication,author,date,year,month,url,content
1,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,None,WASHINGTON — Congressional Republicans have...


In [3]:
#Creando un dataframe que contenga los contedidos de las publicaciones hechas y limpiando el contenido de caracteres especiales.
df1['_cleaned'] = [re.sub(r"[^a-zA-Z0-9]+",' ', str(x)) for x in df1['_c9']]
df1.head()


,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_cleaned
0,None,id,title,publication,author,date,year,month,url,content,content
1,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,None,WASHINGTON — Congressional Republicans have...,WASHINGTON Congressional Republicans have a ne...
2,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,None,"After the bullet shells get counted, the blood...",After the bullet shells get counted the blood ...
3,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,None,"When Walt Disney’s “Bambi” opened in 1942, cri...",When Walt Disney s Bambi opened in 1942 critic...
4,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,None,"Death may be the great equalizer, but it isn’t...",Death may be the great equalizer but it isn t ...


In [4]:
#Para verificar que el procesamiento ocurrio correctamente, haremos un word cloud usando la libreria wordcloud para obtener una representacion visual de las palabras más comunes, esta es la clave para enteder la data y que vamos en el buen camino para el entrenamiento de modelos


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
count_vectorizer = CountVectorizer(stop_words='english')
# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(df1['_cleaned'])

In [6]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
 
# Helper function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
# Tweak the two parameters below
number_topics = 5
number_words = 10
# Create and fit the LDA model
lda = LDA()
lda.fit(count_data)
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

Topics found via LDA:

Topic #0:
trump clinton said donald campaign republican president people hillary 2016

Topic #1:
said isis military state attack syria islamic united war security

Topic #2:
said country european china world government europe people minister party

Topic #3:
company million new year business 000 said companies billion percent

Topic #4:
trump said mr president clinton news house obama white russia

Topic #5:
like people just don know think said time going ve

Topic #6:
said court law federal states state health president government house

Topic #7:
said mr family school new ms women children students years

Topic #8:
said people like water new says study dr health just

Topic #9:
said police told according officers cnn people man reported city